# Generate Website

This notebook generates the website for the dataset.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import DepKind, VersionGroup, DepReport, LinuxImage, DATA_PATH, Dep
from typing import Iterator, List
from pathlib import Path


WEBSITE_URL = "https://depsurf.github.io"
GROUPS = [VersionGroup.REGULAR, VersionGroup.ARCH, VersionGroup.FLAVOR]


def get_report_kind_path(kind: DepKind):
    return DATA_PATH / f"website-{kind.value.lower()}"


def get_report_json_path(dep: Dep):
    return (
        get_report_kind_path(dep.kind)
        / dep.name.replace("_", "").lower()[0]
        / f"{dep.name}.json"
    )


def get_report_url(dep: Dep):
    return f"{WEBSITE_URL}/{get_report_json_path(dep).relative_to(DATA_PATH).with_suffix('.html')}"


## Generate JSON Reports

In [3]:
def test_report(dep: Dep):
    json_path = get_report_json_path(dep)
    if json_path.exists():
        return DepReport.from_dump(json_path)
    else:
        return DepReport.from_groups(dep, GROUPS)


dep = DepKind.FUNC("__blk_account_io_done")
dep = DepKind.FUNC("__page_cache_release")
dep = DepKind.STRUCT("task_struct")
dep = DepKind.STRUCT("fs_context")
dep = DepKind.STRUCT("inode")
dep = DepKind.FUNC("vfs_rename")
dep = DepKind.FUNC("init_once")
dep = DepKind.FUNC("vfs_read")
dep = DepKind.STRUCT("xfrm_policy_afinfo")
dep = DepKind.STRUCT("xfrm_sec_ctx")
dep = DepKind.STRUCT("address_space_operations")
dep = DepKind.FUNC("blk_account_io_start")

# test_report(dep)


In [10]:
def get_deps_names(img: LinuxImage, kind: DepKind) -> Iterator[str]:
    if kind == DepKind.FUNC:
        return (func.name for func in img.func_groups.iter_funcs())
    elif kind == DepKind.STRUCT:
        return iter(img.struct_types.data)
    elif kind == DepKind.LSM:
        return iter(img.lsm_hooks)
    elif kind == DepKind.TRACEPOINT:
        return iter(img.tracepoints.data)
    elif kind == DepKind.KFUNC:
        return iter(img.kfuncs)
    else:
        raise ValueError(f"Unknown kind: {kind}")


def dump_reports(kinds: List[DepKind]):
    for dep in sorted(
        set(
            kind(name)
            for group in GROUPS
            for v in group
            for kind in kinds
            for name in get_deps_names(v.img, kind)
        )
    ):
        report = DepReport.from_groups(dep, GROUPS)
        report.dump_json(get_report_json_path(dep))


dump_reports(
    [
        # DepKind.FUNC,
        # DepKind.STRUCT,
        # DepKind.LSM,
        # DepKind.TRACEPOINT,
    ]
)

## Generate Markdown Reports

In [5]:
def dump_markdowns(kinds: List[DepKind]):
    for kind in kinds:
        paths = get_report_kind_path(kind).rglob("*.json")
        for json_path in paths:
            report = DepReport.from_dump(json_path)
            report.dump_md(json_path.with_suffix(".md"))


dump_markdowns(
    [
        # DepKind.FUNC,
        # DepKind.STRUCT,
        # DepKind.LSM,
        # DepKind.TRACEPOINT,
    ]
)


## Generate Index Pages

In [9]:
from typing import Iterable

from depsurf import DepKind, IssueEnum
from typing import TextIO


ISSUE_SYMBOLS = {
    IssueEnum.ABSENT: "∅",
    IssueEnum.CHANGE: "Δ",
    IssueEnum.SELECTIVE_INLINE: "S",
    IssueEnum.FULL_INLINE: "F",
    IssueEnum.DUPLICATE: "D",
    IssueEnum.COLLISION: "C",
    IssueEnum.TRANSFORMATION: "T",
}

ISSUE_NAMES = {
    IssueEnum.ABSENT: "Absence",
    IssueEnum.CHANGE: "Change",
    IssueEnum.SELECTIVE_INLINE: "Selective Inline",
    IssueEnum.FULL_INLINE: "Full Inline",
    IssueEnum.DUPLICATE: "Duplication",
    IssueEnum.COLLISION: "Collision",
    IssueEnum.TRANSFORMATION: "Transformation",
}


def print_row(report_path: Path, group_only: bool, file: TextIO):
    report = DepReport.from_dump(report_path)
    dep = report.dep
    name = f"[`{dep.name}`]({get_report_url(dep)})"
    if group_only:
        grouped_issues = {group: set() for group in GROUPS}
        for (group, _), issues in report.issues_dict.items():
            grouped_issues[group].update(issues)
        rows = [
            ", ".join([ISSUE_NAMES[e] for e in grouped_issues[group]])
            for group in GROUPS
        ]
    else:
        rows = [
            ",".join([ISSUE_SYMBOLS[e] for e in issues])
            for issues in report.issues_dict.values()
        ]
    print("|".join(["", name, *rows, ""]), file=file)


def print_header(group_only: bool, file: TextIO):
    if group_only:
        col_names = [g for g in GROUPS]
    else:
        col_names = [g.to_str(v) for g in GROUPS for v in g]
    print("| Name | " + " | ".join(col_names) + " |", file=file)
    print("| " + " | ".join(["-"] * (len(col_names) + 1)) + " |", file=file)


def print_index(
    report_paths: Iterable[Path], file_path: Path, group_only: bool = False
):
    with open(file_path, "w") as f:
        kind = file_path.parent.name
        print(f"# {kind}", file=f)
        print_header(group_only, file=f)
        for report_path in sorted(
            report_paths, key=lambda d: d.stem.replace("_", "").lower()
        ):
            print_row(report_path, group_only=group_only, file=f)
    print(f"Saved {file_path}", flush=True)


def dump_index(kinds: List[DepKind]):
    for kind in kinds:
        website_path = DATA_PATH / "website"
        kind_path = get_report_kind_path(kind)
        subdirs = sorted(d for d in kind_path.iterdir() if d.is_dir())
        if kind in [DepKind.FUNC, DepKind.STRUCT]:
            for subdir in subdirs:
                print_index(
                    report_paths=subdir.glob("*.json"),
                    file_path=website_path / f"{kind.value.lower()}-{subdir.stem}.md",
                    group_only=(kind == DepKind.FUNC),
                )
        else:
            print_index(
                report_paths=[p for d in subdirs for p in d.glob("*.json")],
                file_path=website_path / f"{kind.value.lower()}.md",
            )


dump_index(
    [
        # DepKind.FUNC,
        # DepKind.STRUCT,
        # DepKind.LSM,
        # DepKind.TRACEPOINT,
    ]
)